<a href="https://colab.research.google.com/github/longjae/llm_tuning_study/blob/master/gemma_ko_2b_instruction_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 환경 설정

## 1-1. 필수 라이브러리 설치

In [ ]:
!pip3 install -q -U transformers==4.38.2
!pip3 install -q -U datasets==2.18.0
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.9.0
!pip3 install -q -U trl==0.7.11
!pip3 install -q -U accelerate==0.27.2

## 1-2. Import Modules

In [ ]:
import torch
from datasets import Dataset, load_dataset # hugging face 데이터셋 허브에서 데이터 로드
# AutoTokenizer, AutoModelForCausalLM: 자동으로 사전 학습된 tokenizer와 언어 모델을 불러옴
# BitsAndBytesConfig: 모델의 퀀타이제이션을 설정하여 모델의 메모리 사용량을 줄이는데 사용
# Pipeline: hugging face의 inference pipeling을 통해 특정 작업에 맞는 모델을 손쉽게 사용
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments
from peft import LoraConfig, PeftModel # 모델을 효율적으로 파인튜닝
from trl import SFTTrainer # Supervised Fine-Tuning을 위한 트레이너

## 1-3. HugggingFace

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

# 2. DataSet

## 2-1. 데이터 로드

In [ ]:
from datasets import load_dataset
dataset = load_dataset("jihye-moon/LawQA-Ko")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


## 2-2. 데이터 탐색

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'precedent', 'answer'],
        num_rows: 14819
    })
})

## 2-3. 데이터셋 예시

In [ ]:
dataset['train'][0]

{'question': '최저임금제도란 무엇이며, 최저임금액은 어떻게 결정되는지요?',
 'precedent': '',
 'answer': '최저임금제도란 국가가 임금액의 최저한도를 정하여 사용자에게 이를 준수하도록 강제하는 제도를 말합니다. 그러므로 최저임금액이 결정·고시되면 사용자는 근로자와 합의하여 최저임금액보다 낮은 임금을 지급하기로 약정하더라도 그것은 당연히 무효가 되며, 고용노동부장관이 고시한 최저임금액 이상을 지급하여야 합니다(최저임금법 제6조). 최저임금제도의 적용대상은 근로자를 사용하는 모든 사업 또는 사업장에 적용되며, 상용근로자 뿐만 아니라 임시근로자나 일용근로자, 시간제근로자 등 모든 근로자에게 적용됩니다. 다만, 동거의 친족만을 사용하는 사업과 가사사용인에 대하여는 적용하지 아니하고, 선원법의 적용을 받는 선원 및 선원을 사용하는 선박의 소유자에 대하여는 이를 적용하지 아니합니다(같은 법 제3조). 그리고 수습사용 중에 있는 자로서 수습사용한 날부터 3월 이내인 근로자는 시간급 최저임금액의 90%를 지급할 수 있고, 사용자가 고용노동부장관의 인가를 받은 감시 또는 단속적으로 근로에 종사하는 자(수위, 경비원, 자가용운전기사 등)는 시간급 최저임금액의 80%를 지급할 수 있습니다(같은 법 제5조 제2항, 같은 법 시행령 제3조).그러나 같은 법 시행령 제6조는 사용자는 고용노동부장관의 인가를 받아 ‘근로자의 정신 또는 신체의 장애가 당해 근로자를 종사시키고자 하는 업무의 수행에 직접적으로 현저한 지장을 주는 것이 명백하다고 인정되는 자’에 대하여는 최저임금의 적용을 제외할 수 있도록 규정하고 있습니다.고용노동부장관은 매년 3월 31일까지 근로자위원, 사용자위원, 공익위원 등으로 구성된 최저임금심의위원회에 최저임금에 관한 심의를 요청하여야 하고, 동 위원회에서는 근로자의 생계비, 유사근로자의 임금, 노동생산성 및 소득분배율 등을 고려하여 최저임금안을 심의하며, 심의위원회로부터 최저임금안을 제출받은 때에는 지체없이 사업의 종류별 최저임금안 및

# 3. Pre-trained Model(Gemma base)

## 3-1. 모델 로드

`device_map` 파라미터를 통해 모델을 로드할 디바이스를 지정한다.

`{"":0}`은 모델의 모든 파라미터를 첫 번째 GPU(즉, GPU 0)에 로드하라는 의미

`add_special_tokens` 파라미터는 특별 토큰(special tokens)을 추가할지 여부를 결정한다. 특별 토큰은 모델에 따라 다르지만, 일반적으로 문장의 시작과 끝을 나타냄.

In [ ]:
PRE_TRAINED_MODEL = "beomi/gemma-ko-2b"

# 사전 학습된 causal language 모델을 불러옴
model = AutoModelForCausalLM.from_pretrained(PRE_TRAINED_MODEL, device_map={"": 0})
# 사전 학습된 토크나이저 불러오기 (특수 토큰 추가)
tokenizer = AutoTokenizer.from_pretrained(PRE_TRAINED_MODEL, add_special_tokens=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

## 3-2. 추론 수행

In [ ]:
pipe = pipeline(
    "text-generation",  # 텍스트 생성을 위한 파이프라인 설정
    model=model,        # 사전 학습된 모델을 사용
    tokenizer=tokenizer # 해당 모델과 호환되는 토크나이저 사용
)

In [ ]:
prompt = "최저임금제도에 대해 알려줘"


`do_sample` 파라미터를 통해 텍스트를 생성할 때 확률적 샘플링을 사용할지 여부를 결정합니다. True로 설정하면 확률에 기반하여 토큰을 샘플링하여 다양한 출력을 생성할 수 있습니다. False로 설정하면 그리디 서치(가장 높은 확률의 토큰을 선택)를 사용합니다.

`temperature` 파라미터를 통해 샘플링 시 확률 분포를 제어하는 매개변수입니다. 값이 낮을수록 확률 분포가 더욱 집중되며, 모델이 더 보수적으로 예측하게 됩니다. 값이 높을수록 확률 분포가 넓어져 더 다양하고 창의적인 출력을 생성할 수 있습니다.
온도가 낮을수록  보수적이고 일관된 출력을 생성하며, 온도가 높을수록 더 다양한 출력을 생성할 가능성이 높습니다.

`top_k` 매 샘플링 단계에서 고려할 상위 k개의 토큰만 사용합니다. 상위 50개의 후보 토큰 중에서만 선택하여 다음 토큰을 생성합니다. 희귀하고 덜 관련된 토큰을 제외하고, 상위 후보 중에서 샘플링하여 더 합리적인 출력을 생성하도록 합니다.

`top_p` 누적 확률이 p(0.95) 이상이 되는 상위 토큰 집합에서 샘플링합니다. 이 방법을 Nucleus Sampling이라고도 합니다. 확률 질량의 95%를 차지하는 상위 토큰을 선택하여 다음 토큰을 생성합니다. 이로 인해 유연한 샘플링이 가능하며, 확률이 매우 낮은 토큰을 배제하여 출력을 향상시킵니다.

`repetition_penalty` 반복 페널티를 적용하여 동일한 단어가 반복되지 않도록 합니다. 1보다 큰 값을 사용하면 반복되는 단어에 패널티를 부여합니다. 출력을 더 다양하게 만들어 반복을 방지합니다. 값이 클수록 더 강한 패널티가 적용됩니다.

`add_special_tokens` 특별 토큰(special tokens)을 추가할지 여부를 결정합니다. 예를 들어, 문장의 시작과 끝을 나타내는 토큰 등을 추가합니다.

In [ ]:
outputs = pipe(
    prompt,                   # 입력 프롬프트(생성할 텍스트의 시작 부분)
    do_sample=True,           # 샘플링을 통해 다양한 텍스트 생성
    temperature=0.2,          # 온도 조절 (값이 낮을수록 보수적인 텍스트 생성, 높을수록 다양성 증가)
    top_k=50,                 # top-k 샘플링, 상위 50개의 예측 토큰 중에서 선택
    top_p=0.95,               # top-p 샘플링, 확률이 상위 95%에 해당하는 토큰들 중에서 선택
    repetition_penalty=1.2,   # 반복 패널티, 동일한 패턴이 반복되지 않도록 패널티 적용
    add_special_tokens=True   # 특수 토큰 추가 (모델이 필요로 하는 경우)
)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
outputs[0]

{'generated_text': '최저임금제도에 대해 알려줘요! 1. 최저임'}

# 4. Instruction Tuning

In [ ]:
import locale
locale.getpreferredencoding = lambda: 'UTF-8'

!nvidia-smi

Tue Oct  1 11:11:29 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0              62W / 400W |  10719MiB / 40960MiB |     47%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

## 4-1. 학습 프롬프트 생성

In [ ]:
train_data = dataset['train']
train_data

Dataset({
    features: ['question', 'precedent', 'answer'],
    num_rows: 14819
})

In [ ]:
train_data[0]

{'question': '최저임금제도란 무엇이며, 최저임금액은 어떻게 결정되는지요?',
 'precedent': '',
 'answer': '최저임금제도란 국가가 임금액의 최저한도를 정하여 사용자에게 이를 준수하도록 강제하는 제도를 말합니다. 그러므로 최저임금액이 결정·고시되면 사용자는 근로자와 합의하여 최저임금액보다 낮은 임금을 지급하기로 약정하더라도 그것은 당연히 무효가 되며, 고용노동부장관이 고시한 최저임금액 이상을 지급하여야 합니다(최저임금법 제6조). 최저임금제도의 적용대상은 근로자를 사용하는 모든 사업 또는 사업장에 적용되며, 상용근로자 뿐만 아니라 임시근로자나 일용근로자, 시간제근로자 등 모든 근로자에게 적용됩니다. 다만, 동거의 친족만을 사용하는 사업과 가사사용인에 대하여는 적용하지 아니하고, 선원법의 적용을 받는 선원 및 선원을 사용하는 선박의 소유자에 대하여는 이를 적용하지 아니합니다(같은 법 제3조). 그리고 수습사용 중에 있는 자로서 수습사용한 날부터 3월 이내인 근로자는 시간급 최저임금액의 90%를 지급할 수 있고, 사용자가 고용노동부장관의 인가를 받은 감시 또는 단속적으로 근로에 종사하는 자(수위, 경비원, 자가용운전기사 등)는 시간급 최저임금액의 80%를 지급할 수 있습니다(같은 법 제5조 제2항, 같은 법 시행령 제3조).그러나 같은 법 시행령 제6조는 사용자는 고용노동부장관의 인가를 받아 ‘근로자의 정신 또는 신체의 장애가 당해 근로자를 종사시키고자 하는 업무의 수행에 직접적으로 현저한 지장을 주는 것이 명백하다고 인정되는 자’에 대하여는 최저임금의 적용을 제외할 수 있도록 규정하고 있습니다.고용노동부장관은 매년 3월 31일까지 근로자위원, 사용자위원, 공익위원 등으로 구성된 최저임금심의위원회에 최저임금에 관한 심의를 요청하여야 하고, 동 위원회에서는 근로자의 생계비, 유사근로자의 임금, 노동생산성 및 소득분배율 등을 고려하여 최저임금안을 심의하며, 심의위원회로부터 최저임금안을 제출받은 때에는 지체없이 사업의 종류별 최저임금안 및

In [ ]:
def generate_prompt(example):
    output_texts = []
    for i in range(len(example['question'])):
        # 프롬프트 형식 지정
        prompt = f"### Instruction: {example['question'][i]}\n\n### Response: {example['answer'][i]}<eos>"
        output_texts.append(prompt)
    return output_texts

In [ ]:
print(generate_prompt(train_data[:1])[0])

### Instruction: 최저임금제도란 무엇이며, 최저임금액은 어떻게 결정되는지요?

### Response: 최저임금제도란 국가가 임금액의 최저한도를 정하여 사용자에게 이를 준수하도록 강제하는 제도를 말합니다. 그러므로 최저임금액이 결정·고시되면 사용자는 근로자와 합의하여 최저임금액보다 낮은 임금을 지급하기로 약정하더라도 그것은 당연히 무효가 되며, 고용노동부장관이 고시한 최저임금액 이상을 지급하여야 합니다(최저임금법 제6조). 최저임금제도의 적용대상은 근로자를 사용하는 모든 사업 또는 사업장에 적용되며, 상용근로자 뿐만 아니라 임시근로자나 일용근로자, 시간제근로자 등 모든 근로자에게 적용됩니다. 다만, 동거의 친족만을 사용하는 사업과 가사사용인에 대하여는 적용하지 아니하고, 선원법의 적용을 받는 선원 및 선원을 사용하는 선박의 소유자에 대하여는 이를 적용하지 아니합니다(같은 법 제3조). 그리고 수습사용 중에 있는 자로서 수습사용한 날부터 3월 이내인 근로자는 시간급 최저임금액의 90%를 지급할 수 있고, 사용자가 고용노동부장관의 인가를 받은 감시 또는 단속적으로 근로에 종사하는 자(수위, 경비원, 자가용운전기사 등)는 시간급 최저임금액의 80%를 지급할 수 있습니다(같은 법 제5조 제2항, 같은 법 시행령 제3조).그러나 같은 법 시행령 제6조는 사용자는 고용노동부장관의 인가를 받아 ‘근로자의 정신 또는 신체의 장애가 당해 근로자를 종사시키고자 하는 업무의 수행에 직접적으로 현저한 지장을 주는 것이 명백하다고 인정되는 자’에 대하여는 최저임금의 적용을 제외할 수 있도록 규정하고 있습니다.고용노동부장관은 매년 3월 31일까지 근로자위원, 사용자위원, 공익위원 등으로 구성된 최저임금심의위원회에 최저임금에 관한 심의를 요청하여야 하고, 동 위원회에서는 근로자의 생계비, 유사근로자의 임금, 노동생산성 및 소득분배율 등을 고려하여 최저임금안을 심의하며, 심의위원회로부터 최저임금안을 제출받은 때에는 지체없이 사업의 종류별 최저임금안 및 적용사업의 범위를 고시하

## 4-2. QLoRA 설정

`quantization_config` 파라미터를 통해 모델 양자화(quantization) 설정을 지정한다. bnb_config는 양자화 설정이 담긴 변수로, 모델의 메모리 사용량을 줄이고 성능을 최적화하는 데 사용됨.

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,                      # 4비트 양자화를 활성화
    bnb_4bit_quant_type="nf4",              # NF4 양자화 방식 사용 (효율적인 4비트 표현)
    bnb_4bit_compute_dtype=torch.float16    # 연산에 float16 데이터 타입 사용
)

BASE_MODEL = "beomi/gemma-ko-2b"

# 사전 학습된 causal language 모델을 양자화된 설정으로 불러오기
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    device_map={"": 0},                     # 모델을 GPU 0번 장치에 할당
    quantization_config=bnb_config          # 양자화 설정 적용
)

# 토크나이저 불러오기 및 설정
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, add_special_tokens=True)
tokenizer.padding_side = 'right'            # 패딩을 오른쪽에 추가

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

`LoRA (Low-Rank Adaptation)` 설정을 통해 자연어 처리 모델의 파인튜닝을 위해 사용. 특정 파라미터에 대해 저차원 근사화를 사용하여 메모리와 계산 효율성을 높이는 기법

In [ ]:
lora_config = LoraConfig(
    r=6, # 저차원(rank) 크기, LoRA의 병목 차원 크기를 설정
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM", # 작업 유형을 Causal Language Modeling으로 설정
)

## 4-3. Trainer

`max_seq_length` 모델이 처리할 수 있는 입력 시퀀스의 최대 길이로 입력 시퀀스가 이 길이를 초과하면 잘리거나 패딩됨

`peft_config=lora_config` LoRA 설정을 지정

`formatting_func=generate_prompt` 입력 데이터를 모델의 입력 형식에 맞게 변환하는 함수로 프롬프트를 생성하는 데 사용됩니다.


`args=TrainingArguments(...)` 학습 설정을 위한 인자들을 포함하는 TrainingArguments 객체

+ `output_dir="outputs"` 학습 결과(모델, 로그 등)를 저장할 디렉토리입니다.

+ `max_steps=3000` 최대 학습 스텝 수

`per_device_train_batch_size=1` 각 디바이스(예: GPU)에서 학습할 배치 크기

`gradient_accumulation_steps=4` 그래디언트 누적 스텝 수로 4 스텝 동안 그래디언트를 누적하여 업데이트를 수행

`optim="paged_adamw_8bit"` 옵티마이저로 paged_adamw_8bit를 사용합니다. 이는 메모리 효율성을 높이기 위해 8비트 정밀도를 사용하는 AdamW 옵티마이저

`warmup_steps=0.03` 학습 초기의 워밍업 스텝 비율입니다. 전체 스텝의 3% 동안 학습률을 점진적으로 증가

`learning_rate=2e-4` 학습률입니다. 학습 속도를 제어하는 중요한 하이퍼파라미터로, 0.0002로 설정되어 있습니다.

`fp16=True` 16비트 부동 소수점(FP16) 혼합 정밀도 학습을 사용합니다. 이를 통해 메모리 사용량을 줄이고 학습 속도를 높일 수 있습니다.

`logging_steps=100` 로그를 기록할 스텝 간격입니다. 매 100 스텝마다 학습 로그를 기록합니다.

`push_to_hub=False` 학습된 모델을 Hugging Face Hub에 업로드하지 않습니다.

`report_to='none'` 로그를 기록할 때 사용할 툴입니다. 여기서는 로그를 외부 툴로 보고하지 않습니다.



In [ ]:
trainer = SFTTrainer(
    model=model,                           # 학습할 모델
    train_dataset=train_data,              # 학습에 사용할 데이터셋
    max_seq_length=512,                    # 입력 시퀀스의 최대 길이
    args=TrainingArguments(                # 학습에 필요한 세부 설정
        output_dir="outputs",              # 학습 결과가 저장될 디렉토리
        num_train_epochs = 3,              # 학습 에포크 수
        max_steps=3000,                    # 학습할 최대 스텝 수
        per_device_train_batch_size=1,     # 각 디바이스(예: GPU)당 학습 배치 크기
        gradient_accumulation_steps=4,     # 4번의 배치마다 경사도 업데이트
        optim="paged_adamw_8bit",          # 8비트 AdamW 옵티마이저 사용
        warmup_steps=0.03,                 # 학습 중 웜업 스텝 비율
        learning_rate=2e-4,                # 학습률
        fp16=True,                         # 16비트 부동소수점 연산을 사용하여 메모리 절약 및 속도 향상
        logging_steps=100,                 # 100 스텝마다 로그 기록
        push_to_hub=False,                 # 모델을 Hugging Face Hub에 푸시하지 않음
        report_to='none',                  # 로그를 기록할 때 외부 서비스에 보고하지 않음
    ),
    peft_config=lora_config,               # LoRA 설정을 통해 효율적 미세 조정
    formatting_func=generate_prompt,       # 입력 데이터를 포맷팅하는 함수
)


Map:   0%|          | 0/14819 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:294: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:450: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:
trainer.train()

Step,Training Loss
100,1.763700
200,1.663500
300,1.656400
400,1.657900
500,1.622800
600,1.605900
700,1.605700
800,1.602800
900,1.598600
1000,1.597400


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in

TrainOutput(global_step=3000, training_loss=1.573169937133789, metrics={'train_runtime': 2497.8552, 'train_samples_per_second': 4.804, 'train_steps_per_second': 1.201, 'total_flos': 5.879210344326758e+16, 'train_loss': 1.573169937133789, 'epoch': 0.81})

4-4. Finetuned Model 저장

In [ ]:
ADAPTER_MODEL = "loar_adapt_it"

trainer.model.save_pretrained(ADAPTER_MODEL)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
!ls -alh loar_adapt_it

total 29M
drwxr-xr-x 2 root root 4.0K Oct  1 11:53 .
drwxr-xr-x 1 root root 4.0K Oct  1 11:53 ..
-rw-r--r-- 1 root root  688 Oct  1 11:53 adapter_config.json
-rw-r--r-- 1 root root  29M Oct  1 11:53 adapter_model.safetensors
-rw-r--r-- 1 root root 5.0K Oct  1 11:53 README.md


gemma-ko-2b 모델과 합쳐 하나의 finetuned 모델로

In [ ]:
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map='auto', torch_dtype=torch.float16)
model = PeftModel.from_pretrained(model, ADAPTER_MODEL, device_map='auto', torch_dtype=torch.float16)

model = model.merge_and_unload() # 어댑터 모델과 기본 모델을 병합하고, 어댑터 관련 메모리를 해제
model.save_pretrained('gemma-ko-2b-it')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
!ls -alh gemma-ko-2b-it

total 4.7G
drwxr-xr-x 2 root root 4.0K Oct  1 11:53 .
drwxr-xr-x 1 root root 4.0K Oct  1 11:53 ..
-rw-r--r-- 1 root root  637 Oct  1 11:53 config.json
-rw-r--r-- 1 root root  132 Oct  1 11:53 generation_config.json
-rw-r--r-- 1 root root 4.7G Oct  1 11:53 model-00001-of-00002.safetensors
-rw-r--r-- 1 root root  65M Oct  1 11:53 model-00002-of-00002.safetensors
-rw-r--r-- 1 root root  14K Oct  1 11:53 model.safetensors.index.json


# 5. Finetuning 모델 추론

In [ ]:
!nvidia-smi

Tue Oct  1 11:53:41 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0              51W / 400W |  18449MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r /content/gemma-ko-2b-it /content/drive/MyDrive/Colab\ Notebooks/LLM/

## 5-1. 모델 로드

In [ ]:
BASE_MODEL = "beomi/gemma-ko-2b"
FINETUNE_MODEL = "./gemma-ko-2b-it"

finetune_model = AutoModelForCausalLM.from_pretrained(FINETUNE_MODEL, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, add_special_tokens=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## 5-2. 모델 추론

In [ ]:
pipe_finetuned = pipeline(
    "text-generation",
    model=finetune_model,
    tokenizer=tokenizer,
    max_new_tokens=512
    )

중요. 프롬프트를 학습시킨 데이터셋과 동일한 포맷으로 작성

In [ ]:
prompt = "최저임금액에 대해 알려줘"
formatted_prompt = f"### Question: {prompt}\n\n### Response:"

outputs = pipe_finetuned(
    formatted_prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    repetition_penalty=1.2,
    add_special_tokens=True
)
print(outputs[0]["generated_text"][len(formatted_prompt):])

 최저임금은 근로자의 임금을 보장하기 위한 법률적 기준입니다.


In [ ]:
prompt = "근로기준법이 무엇인가요?"
formatted_prompt = f"### Question: {prompt}\n\n### Response:"

outputs = pipe_finetuned(
    formatted_prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    repetition_penalty=1.2,
    add_special_tokens=True
)
print(outputs[0]["generated_text"][len(formatted_prompt):])

 근로기준법은 사업주와 노동자 사이의 임금, 휴업급여 및 상해보상 등에 관한 사항을 규정하고 있습니다.


In [ ]:
prompt = "노동조합활동중에 재해가 발생한 경우 산재보상을 받을 수 있는지 여부"
formatted_prompt = f"### Question: {prompt}\n\n### Response:"

outputs = pipe_finetuned(
    formatted_prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    repetition_penalty=1.2,
    add_special_tokens=True
)
print(outputs[0]["generated_text"][len(formatted_prompt):])

 근로기준법 제38조는 “노사협약 또는 단체협약이 정하는 바에 따라서 그 규정의 내용과 방법, 이를 위반하거나 부당하게 변경·폐지하지 못한다.”라고 하여 노사협약 등에서 정하고 있으면 그대로 적용되고, 그렇지 않은 때에는 법령이나 조례(구ㆍ읍ㆍ면ㆍ동장)등으로 정할 것임을 명시하여 있습니다. 또한 동법 시행령 제20조에서는 ‘근로자’란 사업주와 근로자가 상호 직접적인 관계 속에서 일하기로 하는 계약관계에 의해서 고용된 자라 함을 원칙으로 하고 있다고 합니다. 그리고 동법 제41조제5항 및 동법 시행령 제9조 각 호 소급적용대상 중 ‘산업안전보건법 제76조 내지 제80조‘ 에서 정한 사망, 장해, 질병 등의 보상 대상인 ‘질병’, ‘장해’, ‘사망’ 은 모두 근로복지공단의 지원대상입니다.(동법 제41조제5항).
